# Model Pipeline

In [74]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [75]:
import json

# Extract tags into a structured format
def process_tags(tags_str):
    # Handle empty or null values
    if not tags_str or tags_str == '[]' or tags_str == '{}':
        return {}
    
    # Convert the string representation to a Python object
    if isinstance(tags_str, str):
        try:
            # Replace single quotes with double quotes for valid JSON
            tags_obj = json.loads(tags_str.replace("'", '"'))
        except json.JSONDecodeError:
            # Alternative approach if the above fails
            try:
                tags_obj = eval(tags_str)  # Be careful with eval - only use with trusted data
            except:
                return {}  # Return empty dict if parsing fails
    else:
        tags_obj = tags_str  # If it's already a Python object
    
    # Handle the case where tags_obj is a list
    if isinstance(tags_obj, list):
        return {}  # Return empty dictionary for list entries
    
    # Now we're sure tags_obj is a dictionary
    # Normalize tag counts by dividing by the sum
    total = sum(tags_obj.values()) if tags_obj else 1
    return {tag: count/total for tag, count in tags_obj.items()}


In [76]:
# Convert multi-genre strings to one-hot encoded features
def expand_genres_to_columns(df):
    # Assuming genres are stored as comma-separated strings
    # If they're in another format (lists, etc.), adjust the splitting logic
    all_genres = set()
    for genres in df['genre']:
        if isinstance(genres, str):
            all_genres.update([g.strip() for g in genres.split(',')])
    
    # Create binary columns for each genre
    for genre in all_genres:
        column_name = f'genre_{genre.lower().replace(" ", "_")}'
        df[column_name] = df['genre'].apply(
            lambda x: 1 if isinstance(x, str) and genre in x else 0
        )
    
    return df

# function to expand the normalized tags into columns (optimized version)
def expand_tags_to_columns(df):
    # First, collect all unique tags across all games
    all_tags = set()
    for tags_dict in df['normalized_tags']:
        all_tags.update(tags_dict.keys())
    
    # Create column names with 'tags_' prefix and lowercase
    tag_columns = ['tags_' + tag.lower() for tag in all_tags]
    
    # Create a dictionary to hold the tag data
    tag_data = {col: np.zeros(len(df), dtype=float) for col in tag_columns}
    
    # Fill in the values from normalized_tags (much more efficient)
    for idx, tags_dict in enumerate(df['normalized_tags']):
        for tag, value in tags_dict.items():
            tag_data['tags_' + tag.lower()][idx] = value
    
    # Create the tag dataframe all at once
    tag_df = pd.DataFrame(tag_data, index=df.index)
    
    # Concatenate the original dataframe with the tag dataframe
    result_df = pd.concat([df, tag_df], axis=1)
    
    return result_df


In [77]:
def preprocess_data(df):
    print("preprocessing data...")
    # Create a copy to avoid modifying the original TODO use original later
    processed_df = df.copy()
    processed_df = processed_df.drop(['appid', 'average_forever', 'average_2weeks', 'median_forever', 'median_2weeks', 'userscore', 'score_rank', 'languages'], axis=1)
    processed_df = pd.get_dummies(processed_df, columns=['owners', "publisher", "developer"])
    
    # Handle missing values in price
    processed_df['price'] = processed_df['price'].fillna(processed_df['price'].median())
    
    # Handle missing values
    processed_df = processed_df.fillna({'positive': 0, 'negative': 0, 'owners': 0, 'ccu': 0})

    # Normalize tag values
    processed_df['normalized_tags'] = processed_df['tags'].apply(process_tags)
    return processed_df

def engineer_features(df):
    print("Engineering features...")
    # Create derived features
    df['review_ratio'] = df['positive'] / (df['positive'] + df['negative'] + 1)  # Add 1 to avoid division by zero
    df['discount_percentage'] = (df['initialprice'] - df['price']) / (df['initialprice'] + 0.01)

    # Scale the price and positive features
    scaler = StandardScaler()
    df['price_scaled'] = scaler.fit_transform(df[['price']])
    # Scale these - not so much info loss since ccu column contains raw counts
    df['positive_scaled'] = scaler.fit_transform(df[['positive']])
    df['negative_scaled'] = scaler.fit_transform(df[['negative']])

    df = expand_genres_to_columns(df)
    df = expand_tags_to_columns(df)

    df = df.drop(['positive', 'negative', 'discount', 'initialprice', 'normalized_tags', 'tags', 'genre', 'price'], axis=1)
    
    # Could play around with dropping some features here
    return df



In [78]:
def build_similarity_matrix(df, features=None, feature_weights=None):
    """
    - feature_weights: Dictionary mapping feature names to weights
                      If a feature is not in this dict, it will receive an equal share
                      of the remaining weight
    """
    # If no features specified, use all columns in the dataframe
    if features is None:
        features = df.columns.tolist()
    
    # Extract the feature matrix
    feature_matrix = df[features].values
    
    # Initialize weights array
    weights_array = np.ones((1, len(features)))
    
    # Apply weights if provided
    if feature_weights is not None:
        # Calculate total weight already assigned
        total_assigned_weight = sum(weight for feature, weight in feature_weights.items() 
                                   if feature in features)
        
        # Count features without specified weights
        unspecified_features = [f for f in features if f not in feature_weights]
        num_unspecified = len(unspecified_features)
        
        # Calculate weight for each unspecified feature
        remaining_weight = 1.0 - total_assigned_weight
        default_weight = remaining_weight / num_unspecified if num_unspecified > 0 else 0
        
        # Apply weights to each feature
        for i, feature in enumerate(features):
            if feature in feature_weights:
                weights_array[0, i] = feature_weights[feature]
            else:
                weights_array[0, i] = default_weight
    
    # Apply weights to feature matrix
    weighted_matrix = feature_matrix * weights_array
    
    # Calculate cosine similarity using the weighted matrix
    similarity_matrix = cosine_similarity(weighted_matrix)
    
    # Create a DataFrame for easier indexing
    similarity_df = pd.DataFrame(
        similarity_matrix,
        index=df.index,
        columns=df.index
    )
    
    return similarity_df

In [ ]:
def get_recommendations(game_idx, similarity_df, df, features_used, n=5):
    print(f"Getting recommendations for index {game_idx} and name {df['name'].iloc[0]}")
    # Check if the game exists in our data
    if game_idx not in similarity_df.index:
        return f"Game with index {game_idx} not found in the database."
    # Get similarity scores for the game
    similarity_scores = similarity_df.loc[game_idx].sort_values(ascending=False)
    
    # Get top N similar games (excluding the game itself)
    similar_games = similarity_scores.iloc[1:n+1]
    
    # Get details of recommended games
    recommendations = df.loc[similar_games.index]
    
    # Add similarity score to recommendations
    recommendations = recommendations.copy()
    recommendations['similarity_score'] = similar_games.values
    
    # Sort by similarity score
    recommendations = recommendations.sort_values('similarity_score', ascending=False)
    # print(recommendations.head())
    
    return recommendations[features_used]


In [99]:
# Create a recommendation function with pre-loaded data
def recommend_by_index(game_idx, n=5):
    return get_recommendations(game_idx, similarity_df, processed_df, features_to_use, n)

# Alternative function that accepts a game name
def recommend_by_name(game_name, n=5):
    # Find the game index
    if game_name not in processed_df['name'].values:
        return f"Game '{game_name}' not found in the database."
    
    game_idx = processed_df[processed_df['name'] == game_name].index[0]
    print("Game idx from rec by name lookup: ", game_idx)
    return get_recommendations(game_idx, similarity_df, processed_df, features_to_use, n)

In [92]:
df = pd.read_csv("../Data\Top 1000 Steam Games 2023 export 2025-07-09 14-37-02.csv")
# Encode categorical variables
# df = df[:20]

df.head(1)

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\wasadmin\AppData\Local\Temp\2\ipykernel_4916\2524999518.py:1: SyntaxWarning: invalid escape sequence '\T'
  df = pd.read_csv("../Data\Top 1000 Steam Games 2023 export 2025-07-09 14-37-02.csv")


,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,216345,5530,0,"10,000,000 .. 20,000,000",0,0,0,0,999,999,0,"English, French, German, Italian, Spanish - Sp...",Action,10775,"{'Action': 5448, 'FPS': 4862, 'Multiplayer': 3..."


In [93]:
processed_df = preprocess_data(df)

processed_df = engineer_features(processed_df)
# Write processed data to parquet
processed_df.to_parquet('../Data/processed_games.parquet')


# Read in parquet data
processed_df = pd.read_parquet('../Data/processed_games.parquet')

features_to_use = ['review_ratio', 'price_scaled', 'ccu']

# Define weights as a dictionary mapping feature names to weights
feature_weights = {
    'price_scaled': 0.5,  # This assigns 0.1 weight to price_scaled
    'ccu' : 0.2
    # The remaining 0.3 weight will be distributed among other features
}


## Do not use without adjusting similarity matrix code to autofill with higher values
# for col in df.columns:
#     if col.startswith('genre_'):
#         feature_weights[col] = 0.1
#     elif col.startswith('tag_'):
#         feature_weights[col] = 0.1 

# Call the function with the dictionary of weights
similarity_df = build_similarity_matrix(processed_df, features_to_use, feature_weights)
# Write processed data to parquet
similarity_df.to_parquet('../Data/games_similarity_matrix.parquet')


# Read in parquet data
similarity_df = pd.read_parquet('../Data/games_similarity_matrix.parquet')

processed_df.head(1)

preprocessing data...
Engineering features...


,name,ccu,"owners_1,000,000 .. 2,000,000","owners_10,000,000 .. 20,000,000","owners_100,000,000 .. 200,000,000","owners_2,000,000 .. 5,000,000","owners_20,000,000 .. 50,000,000","owners_200,000 .. 500,000","owners_200,000,000 .. 500,000,000","owners_5,000,000 .. 10,000,000",...,tags_asynchronous multiplayer,tags_1980s,tags_party-based rpg,tags_turn-based combat,tags_hardware,tags_inventory management,tags_warhammer 40k,tags_medical sim,tags_shoot 'em up,tags_choose your own adventure
0,Counter-Strike,10775,False,True,False,False,False,False,False,False,...,0.0,0.008254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
len(processed_df)

1000

## Read in Dataframe & Use for Recs

In [104]:
similarity_df = pd.read_parquet('../Data/games_similarity_matrix.parquet')
processed_df = pd.read_parquet('../Data/processed_games.parquet')


# Get recommendations by index
game_idx = 18
print(f"Recommending for game at index {game_idx} with details: \n{df.iloc[game_idx]}")
print(processed_df[features_to_use].iloc[game_idx])

recommendations_by_idx = recommend_by_index(game_idx, n=10)
print(f"Recommendations for game at index {game_idx} based on {features_to_use} with weights {feature_weights}:")
print(recommendations_by_idx)

# Get recommendations by name
game_name = 'Half-Life 2: Episode Two'
recommendations_by_name = recommend_by_name(game_name, n=3)
print(f"\nRecommendations for '{game_name}' based on {features_to_use} with weights {feature_weights}:")
print(recommendations_by_name)

Recommending for game at index 18 with details: 
appid                                                            420
name                                        Half-Life 2: Episode Two
developer                                                      Valve
publisher                                                      Valve
score_rank                                                       NaN
positive                                                       33820
negative                                                        1024
userscore                                                          0
owners                                       5,000,000 .. 10,000,000
average_forever                                                    0
average_2weeks                                                     0
median_forever                                                     0
median_2weeks                                                      0
price                                                 